In [1]:
import pandas as pd

from fairfl.model.preprocess import ACSIncomePreprocess
from fairfl.model.model import ACSIncomeModel
from fairfl.metrics.bias_metrics import BiasType
from fairfl.utils.read_data import DatasetUtils

In [2]:
bias_type = BiasType.race
states = ["MT", "WY", "ID", "VT", "UT", "SD", "ME", "NH", "OR", "ND", "IA", "AK", "HI", "NM", "NE", "CO", "WI", "MN",
          "WA", "WV"]
preprocess = ACSIncomePreprocess()
model = ACSIncomeModel(bias_type=bias_type)
utils = DatasetUtils()

In [3]:
def get_states(train, states):
    train_states = utils.get_selected_states(train, states)
    return train_states

In [5]:
test = pd.read_csv("../dataset/test_preprocessed_label.csv").reset_index(drop=True)
train = pd.read_csv("../dataset/train_preprocessed_label.csv").reset_index(drop=True)

In [6]:
train_states = get_states(train, states)

In [7]:
test2 = test.drop('ST', axis=1)
_ = model.print_model_metrics(train_states, test2)

/home/safir/.virtualenvs/ACIncome_bias_fl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
import pickle

l_model = pickle.load(open("lr_selected.pkl", "rb"))

In [9]:
states_result_dfs = list()

for state in states:
    res = model.print_loaded_model_metrics(test, l_model, state)
    states_result_dfs.append(res)

In [10]:
pd.concat(states_result_dfs).to_excel(f"{bias_type}_global.xlsx", index=False)